In [61]:
import torch
import torch.nn as nn
import numpy as np
from level_replay.algo.dqn import DQN
from level_replay.algo.memory import ReplayMemory


In [65]:
class Args():
    def __init__(self):
        self.atoms = 51
        self.hidden_size = 32
        self.noisy_std = 0.1
        self.device = None
        self.capacity = int(1e6)
        self.history_length = 4
        self.discount = 0.99
        self.multi_step = 3
        self.priority_weight = 0.4  # Initial importance sampling weight β, annealed to 1 over course of training
        self.priority_exponent = 0.5
        #self.t = 0  # Internal episode timestep counter

In [66]:
args = Args()

#dqn = DQN(args, 5)

In [46]:
np.random.seed(100)
x = torch.Tensor(np.random.randn(1, 3, 64, 64))
support = torch.linspace(-10, 10, 51)

In [47]:
value, action, action_log_dist, _ = dqn.act(x, None, None)

In [48]:
(action_log_dist * support).sum(2).argmax(1).item()

3

In [19]:
dqn.act(x, None, None)

(tensor([[[-0.0813, -0.4492, -0.2702, -0.2081, -0.6700,  0.1145, -0.0914,
           -0.3462,  0.1335,  0.2785, -0.2072, -0.4303, -0.0804,  0.7119,
            0.6244,  0.1353,  0.0986,  0.3624,  0.4729,  0.3786, -0.0866,
            0.5251, -0.0224, -0.4950, -0.0459,  0.1321, -0.0317,  0.0941,
            0.2795, -0.3317, -0.2543,  0.0749, -0.2027, -0.1179, -0.6567,
           -0.0717, -0.2332, -0.0304,  0.5374,  0.3512, -0.4912, -0.2774,
           -0.1640,  0.3788,  0.3268, -0.4122, -0.5961,  0.0760, -0.3489,
            0.3175, -0.3561]]], grad_fn=<ViewBackward>),
 tensor(141),
 tensor([[[-4.3150, -4.6173, -4.3687, -3.3920, -4.7240, -4.4574, -3.5289,
           -4.3360, -3.5890, -3.9150, -4.5365, -4.4856, -4.7182, -3.4951,
           -3.5753, -3.9679, -3.5062, -3.4742, -4.0464, -3.6183, -4.2134,
           -3.2196, -3.9132, -4.4452, -4.3700, -3.6183, -4.4350, -3.5689,
           -3.7529, -4.1673, -4.4409, -4.2927, -4.3548, -4.6976, -4.5174,
           -3.8975, -3.6786, -3.6551, -3.

In [60]:
np.zeros((1, 1, 51))

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]]])

In [67]:
mem = ReplayMemory(args, args.capacity)

In [74]:
mem.transitions.data['reward'].shape

(1000000,)